In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!apt install unzip
!unzip /content/gdrive/MyDrive/Dataset.zip -d /content/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Archive:  /content/gdrive/MyDrive/Dataset.zip
  inflating: /content/Dataset/fashion.csv  
   creating: /content/Dataset/images/
  inflating: /content/Dataset/images/10037.jpg  
  inflating: /content/Dataset/images/10039.jpg  
  inflating: /content/Dataset/images/10054.jpg  
  inflating: /content/Dataset/images/10096.jpg  
  inflating: /content/Dataset/images/10097.jpg  
  inflating: /content/Dataset/images/10110.jpg  
  inflating: /content/Dataset/images/10174.jpg  
  inflating: /content/Dataset/images/10180.jpg  
  inflating: /content/Dataset/images/10266.jpg  
  inflating: /content/Dataset/images/10267.jpg  
  inflating: /content/Dataset/images/10268.jpg  
  inflating: /content/Dataset/images/10269.jpg  
  inflating: /content/Dataset/images/10292.jpg  
  inflating: /conte

In [ ]:
import pandas as pd

# Load the dataset
fashion_df = pd.read_csv('/content/Dataset/fashion.csv')

# Preview the dataset
print(fashion_df.head())

   ProductId Gender Category SubCategory ProductType Colour   Usage  \
0      42419  Girls  Apparel     Topwear        Tops  White  Casual   
1      34009  Girls  Apparel     Topwear        Tops  Black  Casual   
2      40143  Girls  Apparel     Topwear        Tops   Blue  Casual   
3      23623  Girls  Apparel     Topwear        Tops   Pink  Casual   
4      47154  Girls  Apparel  Bottomwear      Capris  Black  Casual   

                                  ProductTitle      Image  \
0           Gini and Jony Girls Knit White Top  42419.jpg   
1                Gini and Jony Girls Black Top  34009.jpg   
2  Gini and Jony Girls Pretty Blossom Blue Top  40143.jpg   
3   Doodle Kids Girls Pink I love Shopping Top  23623.jpg   
4             Gini and Jony Girls Black Capris  47154.jpg   

                                            ImageURL  
0  http://assets.myntassets.com/v1/images/style/p...  
1  http://assets.myntassets.com/v1/images/style/p...  
2  http://assets.myntassets.com/v1/images

In [ ]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

# Load ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = base_model.predict(preprocessed_img)
    return features.flatten()

def get_image_paths(folder):
    return glob(folder + '*.jpg')  # Adjust the extension as needed


In [ ]:
image_dir = '/content/Dataset/images/'  # Add paths to your images
image_paths = get_image_paths(image_dir)

# Use batch processing for efficiency
features = []
for image_path in image_paths:
    features.append(extract_features(image_path))
X = np.array(features)


In [ ]:
# Combine the features into a single input string
fashion_df['input_text'] = fashion_df['Category'] + ' ' + fashion_df['SubCategory'] + ' ' + fashion_df['Colour'] + ' ' + fashion_df['ProductType'] + ' ' + fashion_df['Usage']
fashion_df['target_text'] = fashion_df['ProductTitle']

# Split the data into training and validation sets
train_df = fashion_df.sample(frac=0.9, random_state=42)
val_df = fashion_df.drop(train_df.index)


In [ ]:
!pip install transformers


In [ ]:
!pip install transformers[torch] accelerate


In [ ]:
!pip install transformers[torch] -U
!pip install accelerate -U



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.9 MB/s eta 0:00:00


In [ ]:
from transformers import DataCollatorForLanguageModeling, GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, texts, targets, max_length=128):
        self.tokenizer = tokenizer
        self.texts = texts
        self.targets = targets
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]
        input_dict = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length)
        target_dict = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length)
        input_dict["labels"] = target_dict["input_ids"]
        return input_dict

def load_dataset(train_df, val_df, tokenizer):
    train_dataset = CustomTextDataset(tokenizer, train_df['input_text'].tolist(), train_df['target_text'].tolist())
    val_dataset = CustomTextDataset(tokenizer, val_df['input_text'].tolist(), val_df['target_text'].tolist())
    return train_dataset, val_dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained("gpt2")

train_dataset, val_dataset = load_dataset(train_df, val_df, tokenizer)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="/content/Dataset",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    #no_cuda=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

# Save the model
model.save_pretrained("/content/Dataset")


Step,Training Loss
500,1.061500
1000,0.708100
1500,0.668700
2000,0.662300
2500,0.646000
3000,0.630300


In [ ]:
# Generate a description for a new product
def generate_description(details):
    input_text = ' '.join(details) # Example: "Apparel Topwear Red Casual Shirt"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output = model.generate(input_ids)
    description = tokenizer.decode(output[0])
    return description


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the trained model
model = GPT2LMHeadModel.from_pretrained("/content/Dataset")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define a prompt for the model
prompt = "Gini and Jony Girls Knit White Top"

# Encode the prompt and generate text
inputs = tokenizer(prompt, return_tensors="pt", max_length=50, truncation=True)
outputs = model.generate(**inputs, max_length=100, temperature=0.7, top_k=40)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0])

print(generated_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gini and Jony Girls Knit White Topwear Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual Shoes Casual


In [ ]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import Blip2ForConditionalGeneration, Blip2Processor
from torch import nn, optim
import torch

# Define Dataset
class FashionDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform
        self.tokenizer = Blip2Processor.from_pretrained('Salesforce/blip2-opt-2.7b')

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = self.image_dir + self.data_frame.iloc[idx, 0]
        image = Image.open(img_name).convert('RGB')
        if self.transform:
            image = self.transform(image)

        description = self.data_frame.iloc[idx, 1]
        input_ids = self.tokenizer.encode(description, return_tensors="pt", max_length=50, truncation=True).squeeze()

        return {'image': image, 'input_ids': input_ids}

# Image Preprocessing
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create Dataset and DataLoader
train_dataset = FashionDataset(csv_file='/content/Dataset/fashion.csv', image_dir='/content/Dataset/images/', transform=image_transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define Model
model = Blip2ForConditionalGeneration.from_pretrained('Salesforce/blip2-opt-2.7b')

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define Loss and Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_dataloader:
        images = batch['image'].to(device)
        input_ids = batch['input_ids'].to(device)

        # Forward Pass
        # You might need to adapt this line depending on the exact model and task
        outputs = model(input_ids=input_ids, pixel_values=images)
        loss = outputs.loss

        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

# Saving Model
torch.save(model.state_dict(), 'model.pth')

# Inference Function
def generate_description(image_path):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image_tensor = image_transform(image).unsqueeze(0).to(device)
    prediction = model.generate(image_tensor, max_length=16)
    processor = Blip2Processor.from_pretrained('Salesforce/blip2-opt-2.7b')
    description = processor.decode(prediction[0], skip_special_tokens=True)
    return description




In [ ]:
# Example Usage
description = generate_description('/content/Dataset/images/sample.jpg')
print(description)

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

# Define Dataset
class FashionDataset(Dataset):
    def __init__(self, csv_file):
        self.data_frame = pd.read_csv(csv_file)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        row = self.data_frame.iloc[idx]
        input_text = ' '.join([str(row['Gender']), str(row['Category']), str(row['SubCategory']), str(row['ProductType']), str(row['Colour']), str(row['Usage'])])
        description = str(row['ProductTitle'])

        input_ids = self.tokenizer.encode(input_text, return_tensors="pt", max_length=50, truncation=True).squeeze()
        target_ids = self.tokenizer.encode(description, return_tensors="pt", max_length=50, truncation=True).squeeze()

        return {'input_ids': input_ids, 'target_ids': target_ids}

# Create Dataset and DataLoader
train_dataset = FashionDataset(csv_file='/content/Dataset/fashion.csv')
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define Model
class DescriptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = BertModel.from_pretrained('bert-base-uncased')
        self.decoder = nn.GRU(768, 768, batch_first=True)
        self.fc = nn.Linear(768, self.encoder.config.vocab_size)

    def forward(self, input_ids, target_ids=None):
        encoder_outputs = self.encoder(input_ids).last_hidden_state
        if target_ids is not None:
            decoder_outputs, _ = self.decoder(target_ids, encoder_outputs[:, 0:1, :])
            return self.fc(decoder_outputs)
        else:
            # Code for generating descriptions (inference)
            pass

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DescriptionModel()
model = model.to(device)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)

        # Forward Pass
        outputs = model(input_ids, target_ids)
        loss = criterion(outputs[:, :-1].reshape(-1, outputs.size(-1)), target_ids[:, 1:].reshape(-1))

        # Backward Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

# Saving Model
torch.save(model.state_dict(), 'model.pth')
